In [6]:
!pip install faiss-cpu


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.9 MB 6.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/14.9 MB 3.7 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/14.9 MB 3.8 MB/s eta 0:00:04
   ------- -------------------------------- 2.6/14.9 MB 3.7 MB/s eta 0:00:04
   --------- ------------------------------ 3.4/14.9 MB 3.2 MB/s eta 0:00:04
   --------- ------------------------------ 3.7/14.9 MB 3.1 MB/s eta 0:00:04
   ----------- ---------------------------- 4.5/14.9 MB 3.0 MB/s eta 0:00:04
   -------------- ------------------------- 5.2/14.9 MB 3.1 MB/s eta 0:00:04
   --------------- ------------------------ 5.8/14.9 MB 3.0 MB/s eta 0:00:04
   ----------------- ---------------------- 6.6/14.9 MB 3.1 MB/s eta 0:00:03
   -------------------- ------------------- 7.6/14.9 MB 3.3 MB/s eta 0:00:03
   ----------------------- ---------------- 8.7/14.9 MB 3.5 MB/s eta 0:00:02
   ---

In [9]:
!pip install together

In [11]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from together import Together
import os

In [12]:
# Initialize Together AI client (replace with your API key)
together_client = Together(api_key="53b5367ca3a3028837f6786a464abcd57246ecf8dc1c7363889adb1fba0f599d")

In [13]:
# Initialize embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
# Sample text data
sample_text = """
Artificial Intelligence (AI) is transforming industries. It enables machines to learn from data and make decisions.
Machine Learning (ML) is a subset of AI that focuses on algorithms that improve with experience.
Natural Language Processing (NLP) allows machines to understand and generate human language.
Deep Learning uses neural networks to model complex patterns in data.
AI applications include chatbots, recommendation systems, and autonomous vehicles.
"""

In [15]:
# Split text into chunks
chunks = sample_text.strip().split('\n')

# Generate embeddings for chunks
embeddings = embedder.encode(chunks)

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [16]:
def retrieve_context(query, k=2):
    # Encode query
    query_embedding = embedder.encode([query])[0]
    # Search FAISS index
    distances, indices = index.search(np.array([query_embedding]), k)
    # Retrieve relevant chunks
    context = [chunks[idx] for idx in indices[0]]
    return " ".join(context)

def generate_response(query, context):
    # Construct prompt
    prompt = f"Context: {context}\n\nQuery: {query}\n\nAnswer:"
    # Call Together AI API
    response = together_client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content


In [ ]:
def main():
    print("RAG Chatbot: Type 'exit' to quit.")
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            break
        # Retrieve relevant context
        context = retrieve_context(query)
        # Generate response
        response = generate_response(query, context)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()

RAG Chatbot: Type 'exit' to quit.


You:  dog types


Bot:  It seems like your query "dog types" is not directly related to the context provided about Deep Learning and AI applications. However, I can still provide information about different types of dog breeds.

There are various types of dog breeds, generally categorized into several groups based on their appearance, behavior, and historical purpose. Some common groups include:

1. Sporting Group: These breeds are typically energetic, agile, and well-suited for outdoor activities such as hunting, retrieving, and tracking. Examples include Labrador Retrievers, Golden Retrievers, and English Springer Spaniels.

2. Hound Group: Hounds are known for their strong sense of smell and hunting


You:  AI


Bot:  Artificial Intelligence, or AI, is a branch of computer science that aims to create systems capable of performing tasks that would normally require human intelligence. These tasks include learning and adapting to new information, understanding human language, recognizing patterns, solving problems, and making decisions.

AI can be categorized into two main types: Narrow AI, which is designed to perform a narrow task (like facial recognition or internet searches), and General AI, which can perform any intellectual task that a human being can do.

The field of AI is rapidly evolving and has numerous applications in various industries. For instance, in healthcare, AI can help doctors to diagnose diseases more accurately and quickly. In finance, AI can be used
